In [ ]:
import os
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm

def find_band_files(mtl_path):
    """Find all band files associated with the MTL file"""
    base_dir = os.path.dirname(mtl_path)
    mtl_filename = os.path.basename(mtl_path)
    product_id = mtl_filename.split('_MTL')[0]
    
    bands = {}
    for file in os.listdir(base_dir):
        if file.startswith(product_id) and (file.endswith('.TIF') or file.endswith('.tif')):
            band_name = file.split('_')[-1].split('.')[0]
            if band_name.startswith('B'):  # Only get band files
                bands[band_name] = os.path.join(base_dir, file)
    return bands

def read_band(file_path):
    with rasterio.open(file_path) as src:
        band = src.read(1).astype(float)
        profile = src.profile
    return band, profile

# === Environmental Indices Calculations ===
def compute_ndvi(nir, red):
    """Normalized Difference Vegetation Index [-1 to 1]"""
    return np.clip((nir - red) / (nir + red + 1e-10), -1, 1)

def compute_evi(nir, red, blue):
    """Enhanced Vegetation Index [-1 to 1]"""
    return np.clip(2.5 * (nir - red) / (nir + 6*red - 7.5*blue + 1), -1, 1)

def compute_savi(nir, red, L=0.5):
    """Soil Adjusted Vegetation Index [-1 to 1]"""
    return np.clip((nir - red) / (nir + red + L) * (1 + L), -1, 1)

def compute_ndmi(nir, swir):
    """Normalized Difference Moisture Index [-1 to 1]"""
    return np.clip((nir - swir) / (nir + swir + 1e-10), -1, 1)

def compute_ndbi(swir, nir):
    """Normalized Difference Built-up Index [-1 to 1]"""
    return np.clip((swir - nir) / (swir + nir + 1e-10), -1, 1)

def compute_bsi(swir, red, nir, blue):
    """Bare Soil Index [-1 to 1]"""
    return np.clip(((swir + red) - (nir + blue)) / ((swir + red) + (nir + blue) + 1e-10), -1, 1)

def compute_carbon(ndvi, lst):
    """Improved Carbon Storage Proxy [0 to 1]"""
    lst_norm = np.clip((lst - (-20)) / (60 - (-20)), 0, 1)
    return np.clip((1 - ndvi) * (1 - lst_norm), 0, 1)

def compute_methane(ndvi, ndbi):
    """Methane Emission Proxy [0 to 1] - Labeled as Urban Stress Index"""
    return np.clip((ndbi + (1 - ndvi)) / 2, 0, 1)

def compute_moisture(ndvi, ndmi):
    """Atmospheric Moisture Proxy [-1 to 1]"""
    return np.clip((ndvi + ndmi) / 2, -1, 1)

def compute_soil_type(bsi, ndvi):
    """Classify soil type based on BSI and NDVI"""
    # 0: Sandy, 1: Loamy, 2: Clayey, 3: Organic
    soil_map = np.zeros_like(bsi)
    soil_map[(bsi > 0.5) & (ndvi < 0.2)] = 0  # Sandy
    soil_map[(bsi > -0.2) & (bsi <= 0.5) & (ndvi < 0.3)] = 1  # Loamy
    soil_map[(bsi <= -0.2) & (ndvi < 0.3)] = 2  # Clayey
    soil_map[ndvi >= 0.3] = 3  # Organic
    return soil_map

def compute_lst(band10, metadata_path):
    """Land Surface Temperature (°C) with realistic bounds"""
    def extract_metadata_value(meta_file, key):
        with open(meta_file, 'r') as f:
            for line in f:
                if key in line:
                    return float(line.split('=')[1].strip())
        raise ValueError(f"Metadata key {key} not found")
    
    ML = extract_metadata_value(metadata_path, 'RADIANCE_MULT_BAND_10')
    AL = extract_metadata_value(metadata_path, 'RADIANCE_ADD_BAND_10')
    K1 = extract_metadata_value(metadata_path, 'K1_CONSTANT_BAND_10')
    K2 = extract_metadata_value(metadata_path, 'K2_CONSTANT_BAND_10')
    
    radiance = ML * band10 + AL
    bt_kelvin = K2 / np.log((K1 / radiance) + 1)
    lst_celsius = bt_kelvin - 273.15
    lst_celsius = np.clip(lst_celsius, -20, 60)
    lst_celsius[np.abs(lst_celsius - np.mean(lst_celsius)) > 3*np.std(lst_celsius)] = np.nan
    return lst_celsius

# === Crop Suitability Analysis ===
class CropSuitabilityAnalyzer:
    def _init_(self):
        self.crop_params = {
            'Cashew': {
                'temp_range': (24, 28),
                'moisture_range': (0.3, 0.5),
                'vegetation_range': (0.2, 0.4),
                'soil_range': (0.5, 0.7),
                'preferred_soil': [1, 3],  # Loamy, Organic
                'color': '#FFA500'  # Orange
            },
            'Cassava': {
                'temp_range': (25, 29),
                'moisture_range': (0.4, 0.6),
                'vegetation_range': (0.1, 0.3),
                'soil_range': (0.3, 0.5),
                'preferred_soil': [0, 1],  # Sandy, Loamy
                'color': '#8B4513'  # Brown
            },
            'Tomatoes': {
                'temp_range': (18, 24),
                'moisture_range': (0.6, 0.8),
                'vegetation_range': (0.4, 0.6),
                'soil_range': (0.7, 0.9),
                'preferred_soil': [1, 2],  # Loamy, Clayey
                'color': '#FF6347'  # Tomato red
            },
            'Maize': {
                'temp_range': (21, 27),
                'moisture_range': (0.6, 0.8),
                'vegetation_range': (0.5, 0.7),
                'soil_range': (0.6, 0.8),
                'preferred_soil': [1, 3],  # Loamy, Organic
                'color': '#FFD700'  # Gold
            }
        }
    
    def calculate_suitability(self, indices_dict):
        """Calculate suitability scores for all crops"""
        # Prepare normalized parameters (0-1 scale)
        lst_norm = (indices_dict['LST'] - (-20)) / (60 - (-20))
        moisture = (indices_dict['Moisture'] + 1) / 2
        vegetation = (indices_dict['NDVI'] + 1) / 2
        soil_quality = (indices_dict['Carbon'] + (1 - indices_dict['BSI'])) / 2
        
        # Get soil type compatibility
        soil_type = indices_dict['SoilType']
        soil_compatibility = {}
        
        suitability = {}
        for crop, params in self.crop_params.items():
            # Temperature suitability (Gaussian distribution)
            temp_center = np.mean(params['temp_range'])
            temp_width = (params['temp_range'][1] - params['temp_range'][0]) / 2
            temp_score = np.exp(-0.5 * ((lst_norm - temp_center/60) / (temp_width/60))**2)
            
            # Moisture suitability (triangular distribution)
            moisture_low, moisture_high = params['moisture_range']
            moisture_score = np.where(
                moisture < moisture_low,
                moisture / moisture_low,
                np.where(
                    moisture > moisture_high,
                    (1 - moisture) / (1 - moisture_high),
                    1
                )
            )
            
            # Vegetation suitability
            veg_low, veg_high = params['vegetation_range']
            veg_score = np.clip((vegetation - veg_low) / (veg_high - veg_low), 0, 1)
            
            # Soil suitability
            soil_low, soil_high = params['soil_range']
            soil_score = np.clip((soil_quality - soil_low) / (soil_high - soil_low), 0, 1)
            
            # Soil type compatibility
            soil_comp = np.isin(soil_type, params['preferred_soil']).astype(float)
            
            # Combined score with weights
            suitability[crop] = np.clip(
                (temp_score*0.3 + moisture_score*0.2 + veg_score*0.2 + soil_score*0.2 + soil_comp*0.1),
                0, 1
            )
        
        return suitability

def generate_prediction(suitability, indices):
    """Generate a prediction based on the suitability analysis"""
    # Calculate average suitability scores
    avg_scores = {crop: np.nanmean(score) for crop, score in suitability.items()}
    
    # Find most suitable crop
    best_crop = max(avg_scores.items(), key=lambda x: x[1])
    
    # Analyze environmental conditions
    lst_avg = np.nanmean(indices['LST'])
    moisture_avg = np.nanmean((indices['Moisture'] + 1) / 2)
    soil_type_counts = np.bincount(indices['SoilType'].flatten().astype(int))
    dominant_soil = np.argmax(soil_type_counts)
    soil_types = ['Sandy', 'Loamy', 'Clayey', 'Organic']
    
    # Generate prediction text
    prediction = f"""
    PREDICTION BASED ON ANALYSIS:
    
    The most suitable crop for this area is {best_crop[0]} with an average suitability score of {best_crop[1]:.2f}.
    
    Environmental Conditions:
    - Average Temperature: {lst_avg:.1f}°C
    - Average Moisture: {moisture_avg:.2f}
    - Dominant Soil Type: {soil_types[dominant_soil]}
    
    Recommendations:
    """
    
    # Add crop-specific recommendations
    if best_crop[0] == 'Cashew':
        prediction += "- Cashew trees thrive in warm climates with well-drained soils.\n"
        prediction += "- Ensure proper spacing (7-9m between trees) for optimal growth.\n"
    elif best_crop[0] == 'Cassava':
        prediction += "- Cassava grows well in various soil types but prefers sandy loams.\n"
        prediction += "- Plant cuttings at 1m spacing for optimal yield.\n"
    elif best_crop[0] == 'Tomatoes':
        prediction += "- Tomatoes require consistent moisture and fertile soil.\n"
        prediction += "- Consider drip irrigation for optimal water management.\n"
    elif best_crop[0] == 'Maize':
        prediction += "- Maize performs best in deep, well-drained soils with good organic matter.\n"
        prediction += "- Plant in rows with 75cm spacing for optimal growth.\n"
    
    # Add general agricultural advice
    prediction += "\nGeneral Agricultural Advice:\n"
    prediction += "- Conduct soil tests to verify nutrient levels before planting.\n"
    prediction += "- Consider crop rotation to maintain soil health.\n"
    prediction += "- Monitor weather forecasts for optimal planting and harvesting times.\n"
    
    return prediction

def display_results(indices_dict, suitability):
    """Display all results including tables, maps, and prediction"""
    # Create figure for tables
    fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
    
    # Environmental indices statistics
    env_stats = []
    for name, array in indices_dict.items():
        if name not in ['Profile', 'SoilType']:
            env_stats.append([
                name,
                f"{np.nanmin(array):.2f}",
                f"{np.nanmax(array):.2f}",
                f"{np.nanmean(array):.2f}",
                f"{np.percentile(array[~np.isnan(array)], 25):.2f}",
                f"{np.percentile(array[~np.isnan(array)], 75):.2f}"
            ])
    
    ax1.axis('off')
    table1 = ax1.table(
        cellText=env_stats,
        colLabels=['Environmental Index', 'Min', 'Max', 'Mean', '25th %', '75th %'],
        loc='center',
        cellLoc='center',
        colColours=['#f5f5f5']*6
    )
    table1.auto_set_font_size(False)
    table1.set_fontsize(10)
    table1.scale(1, 1.2)
    ax1.set_title("Environmental Indices Statistics", pad=20)
    
    # Crop suitability statistics
    crop_stats = []
    for crop in suitability:
        score = suitability[crop]
        crop_stats.append([
            crop,
            f"{np.nanmin(score):.2f}",
            f"{np.nanmax(score):.2f}",
            f"{np.nanmean(score):.2f}",
            f"{np.percentile(score[~np.isnan(score)], 25):.2f}",
            f"{np.percentile(score[~np.isnan(score)], 75):.2f}"
        ])
    
    ax2.axis('off')
    table2 = ax2.table(
        cellText=crop_stats,
        colLabels=['Crop', 'Min', 'Max', 'Mean', '25th %', '75th %'],
        loc='center',
        cellLoc='center',
        colColours=['#f5f5f5']*6
    )
    table2.auto_set_font_size(False)
    table2.set_fontsize(10)
    table2.scale(1, 1.2)
    ax2.set_title("Crop Suitability Statistics", pad=20)
    
    plt.tight_layout()
    plt.show()
    
    # Create figure for the suitability maps
    fig2, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    # Plot each crop suitability map
    for i, crop in enumerate(suitability.keys()):
        im = axes[i].imshow(suitability[crop], cmap='RdYlGn', vmin=0, vmax=1)
        axes[i].set_title(f"{crop} Suitability")
        plt.colorbar(im, ax=axes[i], shrink=0.8)
        axes[i].axis('off')
    
    plt.suptitle("Crop Suitability Maps", y=0.98, fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Display soil type map
    fig3, ax = plt.subplots(figsize=(8, 6))
    soil_colors = ['#F4E3AF', '#D2B48C', '#8B4513', '#556B2F']  # Sandy, Loamy, Clayey, Organic
    cmap_soil = LinearSegmentedColormap.from_list('soil_cmap', soil_colors, N=4)
    
    soil_plot = ax.imshow(indices_dict['SoilType'], cmap=cmap_soil, vmin=0, vmax=3)
    ax.set_title("Soil Type Distribution")
    cbar = plt.colorbar(soil_plot, ax=ax, ticks=[0.375, 1.125, 1.875, 2.625])
    cbar.ax.set_yticklabels(['Sandy', 'Loamy', 'Clayey', 'Organic'])
    ax.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Generate and print prediction
    prediction = generate_prediction(suitability, indices_dict)
    print(prediction)

def run_analysis_from_mtl(mtl_path):
    """Run complete analysis from MTL file path"""
    # Load and process bands
    bands = find_band_files(mtl_path)
    print(f"Found {len(bands)} bands: {list(bands.keys())}")
    
    required_bands = {'B2': 'Blue', 'B4': 'Red', 'B5': 'NIR', 'B6': 'SWIR', 'B10': 'Thermal'}
    loaded_bands = {}
    
    for band_code, band_name in required_bands.items():
        if band_code in bands:
            loaded_bands[band_code], profile = read_band(bands[band_code])
            print(f"Loaded {band_name} band ({band_code})")
        else:
            raise ValueError(f"Missing required band: {band_code} ({band_name})")
    
    # Calculate environmental indices
    print("\nCalculating environmental indices...")
    ndvi = compute_ndvi(loaded_bands['B5'], loaded_bands['B4'])
    lst = compute_lst(loaded_bands['B10'], mtl_path)
    bsi = compute_bsi(loaded_bands['B6'], loaded_bands['B4'], loaded_bands['B5'], loaded_bands['B2'])
    
    indices = {
        'NDVI': ndvi,
        'EVI': compute_evi(loaded_bands['B5'], loaded_bands['B4'], loaded_bands['B2']),
        'SAVI': compute_savi(loaded_bands['B5'], loaded_bands['B4']),
        'NDMI': compute_ndmi(loaded_bands['B5'], loaded_bands['B6']),
        'NDBI': compute_ndbi(loaded_bands['B6'], loaded_bands['B5']),
        'BSI': bsi,
        'Carbon': compute_carbon(ndvi, lst),
        'Methane': compute_methane(ndvi, compute_ndbi(loaded_bands['B6'], loaded_bands['B5'])),
        'Moisture': compute_moisture(ndvi, compute_ndmi(loaded_bands['B5'], loaded_bands['B6'])),
        'LST': lst,
        'SoilType': compute_soil_type(bsi, ndvi),
        'Profile': profile
    }
    
    # Perform crop suitability analysis
    print("\nAssessing crop suitability...")
    analyzer = CropSuitabilityAnalyzer()
    suitability = analyzer.calculate_suitability(indices)
    
    # Display results
    display_results(indices, suitability)
    
    return {**indices, 'CropSuitability': suitability}

if _name_ == '_main_':
    mtl_file = r"C:\Users\njoya\OneDrive\Documentos\web2\phamiq\backend\pharmiq\mp.txt"
    
    try:
        results = run_analysis_from_mtl(mtl_file)
        print("\nAnalysis completed successfully!")
    except Exception as e:
        print(f"\nError during analysis: {str(e)}")
    finally:
        plt.show(block=True)